# LangGraph + MCP (Notebook-safe) + Ollama Autonomous Agent

**PLAN → Human Approval → ACT**

- Notebook-safe MCP tools
- No ClientSession
- Fully autonomous & expandable


In [ ]:
!pip install -U langgraph langchain langchain-ollama mcp

In [ ]:
from typing import TypedDict, List, Dict
from langgraph.graph import StateGraph, END
from langchain_ollama import ChatOllama
import json, os

In [ ]:
llm = ChatOllama(
    model="gpt-oss:20b",
    temperature=0
)

## Agent State

In [ ]:
class AgentState(TypedDict):
    goal: str
    plan: List[str]
    approved: bool
    messages: List[Dict]
    done: bool

## MCP Tools (Notebook-safe)

In [ ]:
from mcp.server.fastmcp import FastMCP

BASE_DIR = "/tmp/mcp_workspace"
os.makedirs(BASE_DIR, exist_ok=True)

mcp = FastMCP("filesystem")

@mcp.tool()
def list_dir(path: str = "."):
    return os.listdir(os.path.join(BASE_DIR, path))

@mcp.tool()
def read_file(path: str) -> str:
    return open(os.path.join(BASE_DIR, path)).read()

@mcp.tool()
def write_file(path: str, content: str) -> str:
    with open(os.path.join(BASE_DIR, path), "w") as f:
        f.write(content)
    return "ok"

## MCP Helpers

In [ ]:
def list_tools():
    return list(mcp._tools.keys())

def call_tool(name: str, args: dict):
    return mcp._tools[name](**args)

## Planner Node (PLAN MODE)

In [ ]:
def planner_node(state: AgentState) -> AgentState:
    prompt = f"""
    You are an AI planner.

    Goal:
    {state['goal']}

    Create a clear step-by-step plan.
    Do NOT execute anything.
    """

    plan = llm.invoke(prompt).content
    state['plan'] = [l for l in plan.splitlines() if l.strip()]
    state['done'] = False
    state['messages'] = []
    return state

## Executor Node

In [ ]:
def executor_node(state: AgentState) -> AgentState:
    tools = list_tools()

    prompt = f"""
    Goal: {state['goal']}
    Plan: {state['plan']}
    History: {state['messages']}

    Available tools: {tools}

    Decide ONE next action.

    Respond ONLY in JSON:
    {{"tool": "tool_name | done", "args": {{}}}}
    """

    action = json.loads(llm.invoke(prompt).content)

    if action['tool'] == 'done':
        state['done'] = True
        return state

    state['messages'].append(action)
    return state

## Tool Node

In [ ]:
def tool_node(state: AgentState) -> AgentState:
    action = state['messages'][-1]
    result = call_tool(action['tool'], action.get('args', {}))
    state['messages'].append({'observation': result})
    return state

## Build LangGraph

In [ ]:
graph = StateGraph(AgentState)

graph.add_node('planner', planner_node)
graph.add_node('executor', executor_node)
graph.add_node('tool', tool_node)

graph.set_entry_point('planner')
graph.add_edge('planner', END)

graph.add_conditional_edges(
    'executor',
    lambda s: END if s['done'] else 'tool'
)

graph.add_edge('tool', 'executor')

app = graph.compile()

## RUN PLAN MODE

In [ ]:
state = {
    'goal': 'Read docs and write short notes into summary.txt',
    'plan': [],
    'approved': False,
    'messages': [],
    'done': False
}

state = app.invoke(state)
state['plan']

## USER CLICKS ACT

In [ ]:
state['approved'] = True
state = app.invoke(state)
state['messages']